# Comtradeからのデータ取得法（API&R）
> UNによる解説ページ  
> * [APIの解説](http://comtrade.un.org/data/doc/api/)  
> * [Rから利用する方法の解説](http://comtrade.un.org/data/Doc/api/ex/r)  

## 1. Rの関数（作成）

In [1]:
get.Comtrade <- function(url="http://comtrade.un.org/api/get?"
                         ,maxrec=250000
                         ,type="C"
                         ,freq="A"
                         ,px="HS"
                         ,ps="now"
                         ,r
                         ,p="all"
                         ,rg="all"
                         ,cc="TOTAL"
                         ,fmt="json"
)
{
  string<- paste(url
                 ,"max=",maxrec,"&" #maximum no. of records returned
                 ,"type=",type,"&" #type of trade (c=commodities)
                 ,"freq=",freq,"&" #frequency
                 ,"px=",px,"&" #classification
                 ,"ps=",ps,"&" #time period
                 ,"r=",r,"&" #reporting area
                 ,"p=",p,"&" #partner country
                 ,"rg=",rg,"&" #trade flow
                 ,"cc=",cc,"&" #classification code
                 ,"fmt=",fmt        #Format
                 ,sep = ""
  )
  
  if(fmt == "csv") {
    raw.data<- read.csv(string,header=TRUE)
    return(list(validation=NULL, data=raw.data))
  } else {
    if(fmt == "json" ) {
      raw.data<- fromJSON(file=string)
      data<- raw.data$dataset
      validation<- unlist(raw.data$validation, recursive=TRUE)
      ndata<- NULL
      if(length(data)> 0) {
        var.names<- names(data[[1]])
        data<- as.data.frame(t( sapply(data,rbind)))
        ndata<- NULL
        for(i in 1:ncol(data)){
          data[sapply(data[,i],is.null),i]<- NA
          ndata<- cbind(ndata, unlist(data[,i]))
        }
        ndata<- as.data.frame(ndata)
        colnames(ndata)<- var.names
      }
      return(list(validation=validation,data =ndata))
    }
  }
}

### 1.1. 関数の引数について  
___
#### maxrec
最大データ数.デフォルトは250,000に設定（認証ユーザーの場合の最大数）.ゲストユーザーは50,000まで.  
___
#### type
貿易のタイプ  
* "C":財貿易（デフォルト）
* "S":サービス貿易
___
#### freq
データの周期  
* "A":年次（デフォルト）
* "M":月次
___
#### px
品目分類の種別.デフォルトは"HS"  
* "HS":Harmonized System (HS), as reported (e.g. if data was originally submitted to UN Comtrade in HS1996 then HS1996 is displayed) (full classification code list in JSON, formatted for select2.js, pass "id" to cc parameter) 
* "H0":HS 1992 (full classification code list in JSON, formatted for select2.js, pass "id" to cc parameter) 
* "H1":HS 1996 (full classification code list in JSON, formatted for select2.js, pass "id" to cc parameter) 
* "H2":HS 2002 (full classification code list in JSON, formatted for select2.js, pass "id" to cc parameter) 
* "H3":HS 2007 (full classification code list in JSON, formatted for select2.js, pass "id" to cc parameter) 
* "H4":HS 2012 (full classification code list in JSON, formatted for select2.js, pass "id" to cc parameter) 
* "ST":Standard International Trade Classification (SITC), as reported (e.g. if data was originally submitted to UN Comtrade in SITC Rev. 1 then SITC Rev. 1 is displayed) (full classification code list in JSON, formatted for select2.js, pass "id" to cc parameter) 
* "S1":SITC Revision 1 (full classification code list in JSON, formatted for select2.js, pass "id" to cc parameter) 
* "S2":SITC Revision 2 (full classification code list in JSON, formatted for select2.js, pass "id" to cc parameter) 
* "S3":SITC Revision 3 (full classification code list in JSON, formatted for select2.js, pass "id" to cc parameter) 
* "S4":SITC Revision 4 (full classification code list in JSON, formatted for select2.js, pass "id" to cc parameter) 
* "BEC":Broad Economic Categories (BEC) (full classification code list in JSON, formatted for select2.js, pass "id" to cc parameter) 
___
#### ps
データの取得期間（5期間まで、コンマで区切ってもOK?)
* "now":直近1期間
* "recent":直近5期間
* YYYY:西暦を指定（2013など）
* YYYYMM:年月を指定（月次データの場合）
___
#### r
報告側の地域・国（入力必須、5地域まで、コンマで区切ってもOK?）
* "all":すべての国
* [その他の場合](http://comtrade.un.org/data/cache/reporterAreas.json)
___
#### p
相手側の地域・国（5地域まで、コンマで区切ってもOK?）
* "all":すべての国（デフォルト）
* [その他の場合](http://comtrade.un.org/data/cache/partnerAreas.json)
___
#### rg
貿易のタイプ（コンマで区切ってもOK?)
* "all":すべてのタイプ（デフォルト）
* "1":輸入
* "2":輸出
* "4":再輸入
* "3":再輸出
___
#### cc
品目分類コード（20までなら、コンマで区切ってもOK?)
* "TOTAL":合計値（デフォルト）
* "AG1","AG2","AG3","AG4","AG5","AG6":分類の桁（"AG2"なら2桁分類）
* "ALL":全コード
___
#### fmt
データ・フォーマット
* "json":json形式（デフォルト）
* "csv":csv形式
___
### 1.2. データの使用制限
___
#### リクエスト制限
* 認証ユーザーは1時間10,000リクエストまで
* ゲストユーザーは1時間100リクエストまで
___
#### パラメータの組み合わせの制限
* ps（データの取得期間）, r（報告側の地域・国） and p（相手側の地域・国） are limited to 5 codes each
* Only one of the above codes may use the special "ALL" value.
* Classification codes (cc) are limited to 20 items. "ALL" is always a valid classification code. 
___

## 2. データ取得
### 2.1. JSON形式
> `rjson`ライブラリを予めインストールすること

In [3]:
library("rjson")
s1 <- get.Comtrade(r="842", p="124,484")
s1

,pfCode,yr,period,periodDesc,aggrLevel,IsLeaf,rgCode,rgDesc,rtCode,rtTitle,...,qtAltCode,qtAltDesc,TradeQuantity,AltQuantity,NetWeight,GrossWeight,TradeValue,CIFValue,FOBValue,estCode
1,H4,2015,2015,2015,0,0,1,Import,842,USA,...,NA,,NA,NA,NA,NA,301035596398,NA,NA,0
2,H4,2015,2015,2015,0,0,2,Export,842,USA,...,NA,,NA,NA,NA,NA,279990083749,NA,NA,0
3,H4,2015,2015,2015,0,0,3,Re-Export,842,USA,...,NA,,NA,NA,NA,NA,46422145927,NA,NA,0
4,H4,2015,2015,2015,0,0,1,Import,842,USA,...,NA,,NA,NA,NA,NA,297499549233,NA,NA,0
5,H4,2015,2015,2015,0,0,2,Export,842,USA,...,NA,,NA,NA,NA,NA,236377320352,NA,NA,0
6,H4,2015,2015,2015,0,0,3,Re-Export,842,USA,...,NA,,NA,NA,NA,NA,50441857206,NA,NA,0


### 2.2. CSV形式
> JSONと異なり、validationが出力されない

In [4]:
library("rjson")
s2 <- get.Comtrade(r="842", p="124,484", fmt="csv")
s2

$validation
NULL

$data
  Classification Year Period Period.Desc. Aggregate.Level Is.Leaf.Code
1             H4 2015   2015         2015               0            0
2             H4 2015   2015         2015               0            0
3             H4 2015   2015         2015               0            0
4             H4 2015   2015         2015               0            0
5             H4 2015   2015         2015               0            0
6             H4 2015   2015         2015               0            0
  Trade.Flow.Code Trade.Flow Reporter.Code Reporter Reporter.ISO Partner.Code
1               1     Import           842      USA          USA          124
2               2     Export           842      USA          USA          124
3               3  Re-Export           842      USA          USA          124
4               1     Import           842      USA          USA          484
5               2     Export           842      USA          USA          484
6          

### 2.3. データの繰り返し取得
上記（2.1.か2.2.）を`for`ループなどで繰り返す.